Read file mesh to tensors

In [1]:
import os, random, math, time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset, random_split
from pytorch3d.datasets import ShapeNetCore
from pytorch3d.structures import Meshes, join_meshes_as_batch
from pytorch3d.ops import GraphConv

In [2]:
def collate_meshes_no_textures(batch):
    mesh_list, labels = [], []
    for item in batch:  # ShapeNetCore returns dicts per your class
        verts, faces = item["verts"], item["faces"]
        mesh_list.append(Meshes(verts=[verts], faces=[faces]))
        li = LABEL_TO_IDX.get(item.get("label",""),
             SYNSET_TO_IDX.get(item.get("synset_id",""), -1))
        labels.append(li)
    y = torch.tensor(labels, dtype=torch.long)
    # Hard assert: all labels mapped
    if (y < 0).any() or (y >= NUM_CLASSES).any():
        bad = y[(y < 0) | (y >= NUM_CLASSES)]
        raise RuntimeError(f"Collate produced out-of-range labels: {bad.tolist()}")
    return join_meshes_as_batch(mesh_list), y

In [3]:
import torch.nn as nn
from torch_scatter import scatter_mean

class MeshGCN(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()
        self.g1 = GraphConv(3, 64)
        self.g2 = GraphConv(64, 128)
        self.g3 = GraphConv(128, 128)
        self.head = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    def forward(self, meshes: Meshes):
        x = meshes.verts_packed()
        edges = meshes.edges_packed()
        m_idx = meshes.verts_packed_to_mesh_idx()

        x = torch.relu(self.g1(x, edges))
        x = torch.relu(self.g2(x, edges))
        x = torch.relu(self.g3(x, edges))

        B, D = len(meshes), x.size(1)
        device = x.device
        sums   = torch.zeros((B, D), device=device)
        counts = torch.zeros(B, device=device)
        sums.index_add_(0, m_idx, x)
        counts.index_add_(0, m_idx, torch.ones_like(m_idx, dtype=x.dtype))
        global_feat = sums / counts.clamp_min(1e-6).unsqueeze(-1)

        return self.head(global_feat)


In [4]:
ROOT = "../Dataset/ShapeNetCore"      
VAL_RATIO = 0.2
BATCH_SIZE = 8
NUM_WORKERS = 0
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)

dataset = ShapeNetCore("../Dataset/ShapeNetCore", version=2, load_textures=False)                  
CLASS_NAMES = sorted(dataset.synset_inv.keys())          # e.g. ['airplane','chair',...]
LABEL_TO_IDX = {lbl: i for i, lbl in enumerate(CLASS_NAMES)}
SYNSET_TO_IDX = {dataset.synset_inv[lbl]: i for i, lbl in enumerate(CLASS_NAMES)}
NUM_CLASSES = len(CLASS_NAMES)  
loader = DataLoader(dataset, batch_size=4, collate_fn=collate_meshes_no_textures)

meshes,y = next(iter(loader))
print(meshes)

N = len(dataset)

n_val = int(math.ceil(N*VAL_RATIO))
n_train = N - n_val

train_set, val_set = random_split(dataset, [n_train,n_val], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle = True,
                          num_workers=NUM_WORKERS, collate_fn=collate_meshes_no_textures)

val_loader = DataLoader(val_set,batch_size=BATCH_SIZE, shuffle = False,
                          num_workers=NUM_WORKERS, collate_fn=collate_meshes_no_textures)

c:\Users\esfod\miniconda3\envs\py3d\lib\site-packages\pytorch3d\datasets\shapenet\shapenet_core.py:116: UserWarning: The following categories are included in ShapeNetCore ver.2's official mapping but not found in the dataset location ../Dataset/ShapeNetCore: 03948459, 02871439, 03001627, 04530566, 03938244, 03624134, 04074963, 02924116, 03761084, 04460130, 03593526, 03207941, 03797390, 02958343, 02747177, 03710193, 04330267, 04379243, 02828884, 04554684, 03790512, 03513137, 03759954, 04090263, 02773838, 02880940, 03691459, 03991062, 04225987, 02946921, 04256520, 03085013, 02818832, 04401088, 03337140, 02691156, 02933112, 04004475, 04099429, 02801938, 04468005, 02942699, 03636649, 03261776, 02843684, 03928116, 02954340, 03467517, 02876657, 03325088
  warnings.warn(msg)


In [5]:
print(f"Detected {NUM_CLASSES} classes:")
for i, name in enumerate(CLASS_NAMES):
    print(f"  {i:2d}: {name} ({dataset.synset_inv[name]})")

Detected 5 classes:
   0: bathtub (02808440)
   1: cellphone (02992529)
   2: clock (03046257)
   3: display (03211117)
   4: laptop (03642806)


In [6]:
def accuracy_from_logits (logits, targets):
    preds = logits.argmax(dim=1)
    return (preds == targets).float().mean().item()

In [7]:
def run_epoch(model, loader, optimizer=None, device="cpu", epoch_tag="train"):
    is_train = optimizer is not None
    model.train(is_train)
    crit = nn.CrossEntropyLoss()

    total_loss = total_acc = 0.0
    total_samples = 0
    printed = 0

    for step, (meshes, y) in enumerate(loader):
        # sanity: show per-batch label stats a few times
        if printed < 2:
            print(f"[{epoch_tag}] step {step}: y min={int(y.min())}, max={int(y.max())}, size={len(y)}")
            printed += 1

        meshes = meshes.to(device)
        y = y.to(device)

        with torch.set_grad_enabled(is_train):
            logits = model(meshes)
            loss = crit(logits, y)

        if is_train:
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

        bs = y.size(0)
        total_samples += bs
        total_loss += loss.item() * bs
        total_acc  += (logits.argmax(1) == y).float().sum().item()

    if total_samples == 0:
        print(f"[{epoch_tag}] WARNING: no valid samples accumulated -> returning NaN")
        return float("nan"), float("nan")
    return total_loss / total_samples, total_acc / total_samples


Check for dataset errors ++

In [8]:
from torch.utils.data import DataLoader
from pytorch3d.structures import Meshes, join_meshes_as_batch

# --- Build stable class indexers from the dataset you already created ---
# dataset.synset_dict: {synset_id -> label}; dataset.synset_inv: {label -> synset_id} (already filtered to loaded cats)
CLASS_NAMES = sorted(dataset.synset_inv.keys())  # e.g. ['airplane', 'chair', ...]
LABEL_TO_IDX = {lbl: i for i, lbl in enumerate(CLASS_NAMES)}
SYNSET_TO_IDX = {dataset.synset_inv[lbl]: i for i, lbl in enumerate(CLASS_NAMES)}  # '03001627' -> class idx

def collate_meshes_no_textures(batch):
    mesh_list, labels = [], []
    for item in batch:  # item is a dict per your class
        verts = item["verts"]
        faces = item["faces"]
        mesh_list.append(Meshes(verts=[verts], faces=[faces]))  # drop textures
        # Prefer label string; fallback to synset_id
        li = LABEL_TO_IDX.get(item.get("label", ""), SYNSET_TO_IDX.get(item.get("synset_id", ""), -1))
        labels.append(li)
    return join_meshes_as_batch(mesh_list), torch.tensor(labels, dtype=torch.long)

# --- Pretty print dataset summary ---
print("Total meshes:", len(dataset))
print("Total categories:", len(CLASS_NAMES))
print("Classes (index -> name -> synset):")
for i, name in enumerate(CLASS_NAMES):
    print(f"  {i:2d}: {name}  ({dataset.synset_inv[name]})")

# --- Peek a single sample ---
sample = dataset[0]
print("\nSample[0] keys:", list(sample.keys()))
print("Sample[0] label string:", sample["label"])
print("Sample[0] synset_id:", sample["synset_id"])
print("Sample[0] verts:", sample["verts"].shape, "faces:", sample["faces"].shape)

# --- Pull one batch and verify shapes + label decoding ---
loader_dbg = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_meshes_no_textures)
meshes, y = next(iter(loader_dbg))

print("\nBatch loaded ✅")
print("Batch size (num meshes):", len(meshes))
print("Packed verts:", meshes.verts_packed().shape, "| Packed faces:", meshes.faces_packed().shape)
print("Packed edges:", meshes.edges_packed().shape)
print("Label IDs:", y.tolist())
print("Label names:", [CLASS_NAMES[i] if 0 <= i < len(CLASS_NAMES) else "<?>"
                      for i in y.tolist()])

# --- Extra: quick forward sanity (no grad) if you already defined `model` ---
try:
    with torch.no_grad():
        logits = model(meshes.to(next(model.parameters()).device))
    print("Model forward OK. Logits shape:", tuple(logits.shape))
except Exception as e:
    print("Model forward skipped / error:", repr(e))


Total meshes: 3891
Total categories: 5
Classes (index -> name -> synset):
   0: bathtub  (02808440)
   1: cellphone  (02992529)
   2: clock  (03046257)
   3: display  (03211117)
   4: laptop  (03642806)

Sample[0] keys: ['synset_id', 'model_id', 'verts', 'faces', 'textures', 'label']
Sample[0] label string: cellphone
Sample[0] synset_id: 02992529
Sample[0] verts: torch.Size([11414, 3]) faces: torch.Size([45938, 3])

Batch loaded ✅
Batch size (num meshes): 4
Packed verts: torch.Size([8568, 3]) | Packed faces: torch.Size([33706, 3])
Packed edges: torch.Size([25210, 2])
Label IDs: [1, 0, 2, 3]
Label names: ['cellphone', 'bathtub', 'clock', 'display']
Model forward skipped / error: NameError("name 'model' is not defined")


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MeshGCN(NUM_CLASSES).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4,weight_decay=1e-4)
EPOCHS = 15

best_val_acc = 0

for epoch in range(1,EPOCHS + 1):
    t0 = time.time()
    tr_loss, tr_acc = run_epoch(model, train_loader, optimizer=optimizer, device=device, epoch_tag="train")
    va_loss, va_acc = run_epoch(model, val_loader, optimizer=None, device=device, epoch_tag="val")
    dt = time.time() - t0
    
    print(f"[{epoch:02d}/{EPOCHS}] "
          f"train loss {tr_loss:.4f} acc {tr_acc:.3f} | "
          f"val loss {va_loss:.4f} acc {va_acc:.3f} | {dt:.1f}s")
    if va_acc > best_val_acc:
        best_val_acc = va_acc
        torch.save({"model":model.state_dict(),
                   "val_acc":va_acc}, "meshgcn_best.pt")
        
    print(f"Best val acc: {best_val_acc:.3f}")

[train] step 0: y min=0, max=4, size=8
[train] step 1: y min=0, max=3, size=8


KeyboardInterrupt: 